<h1>working with mnist data set</h1>

In [52]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.ops import rnn_cell
from tensorflow.contrib import rnn

In [53]:
mnist = input_data.read_data_sets('/tmp/data',one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [54]:
hm_epochs = 10
n_classes = 10
batch_size = 128

chunk_size = 28
n_chunks = 28
rnn_size = 128

In [55]:
# height * width
x = tf.placeholder('float',[None,n_chunks,chunk_size])
y = tf.placeholder('float')

In [62]:
def recurrent_neural_network(x):
    # input_data * weights + biases
    layer = {'weights':tf.Variable(tf.random_normal([rnn_size, n_classes])),
                     'biases': tf.Variable(tf.random_normal([n_classes]))}
    x = tf.transpose(x,[1,0,2])
    x = tf.reshape(x, [-1, chunk_size])
    x = tf.split(x,n_chunks,0)
    
    lstm_cell = rnn_cell.BasicLSTMCell(rnn_size)
    outputs,states = rnn.static_rnn(lstm_cell,x,dtype=tf.float32)

    
    output = tf.add(tf.matmul(outputs[-1], layer['weights']) , layer['biases'])
    return output

In [63]:
def train_neural_network(x):
    prediction = recurrent_neural_network(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )
    
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    # cycles feedback + backprops
    #hm_epochs = 10
    
    with tf.Session() as alpha:
        alpha.run(tf.global_variables_initializer())
        
        # training the network
        for e in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                epoch_x = epoch_x.reshape((batch_size,n_chunks,chunk_size))
                _, c = alpha.run([optimizer,cost],feed_dict={x:epoch_x, y:epoch_y})
                epoch_loss +=c
            print('Epoch {} completed out of {} loss: {}'.format(e,hm_epochs,epoch_loss))
        
        
        correct = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct,'float'))
        print('Accuracy: ',accuracy.eval({x:mnist.test.images.reshape((-1,n_chunks,chunk_size)), y:mnist.test.labels}))

In [64]:
train_neural_network(x)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Epoch 0 completed out of 10 loss: 183.91098015010357
Epoch 1 completed out of 10 loss: 55.91682533174753
Epoch 2 completed out of 10 loss: 40.35405230335891
Epoch 3 completed out of 10 loss: 29.826759412884712
Epoch 4 completed out of 10 loss: 24.743336583022028
Epoch 5 completed out of 10 loss: 21.735583344940096
Epoch 6 completed out of 10 loss: 17.807939541060477
Epoch 7 completed out of 10 loss: 16.28584866959136
Epoch 8 completed out of 10 loss: 13.874204866937362
Epoch 9 completed out of 10 loss: 12.454153132042848
Accuracy:  0.9857
